In [7]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing
import scipy.optimize

In [8]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2 import (
    recursive_scan,
    write_excel_xlsx,
    Run_P2_till_Fail,
    Para_init,
    Initialize_mdic_dry,
    Run_Breakin,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
)

In [9]:
Para_dict_All = {
   "Total ageing cycles":[4,],
   "Ageing cycles between RPT":[2,],
   "Update cycles for ageing":[2,],
   "Cycles within RPT":[1,],
   "Ageing temperature":[25,],
   "RPT temperature":[25,],
   "Particle mesh points":[30,],   # Simon uses 30
   #"Exponential mesh stretch":[1.0],
   "Para_Set":[ "Li2023_Coupled",],
   "Model option":[
         {
            "calculate discharge energy":"true",
            "particle": "Fickian diffusion",          
            "SEI":"ec reaction limited",   
            "SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            "particle mechanics":("swelling and cracking", "swelling only"), 
            "loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      },
         ],
   "Inner SEI reaction proportion":[0.5,],
   "Ratio of lithium moles to SEI moles":[ 2,], # I have always been using 1 for solvent consumption model
   "Initial inner SEI thickness [m]":[2.5E-9,],
   "Initial outer SEI thickness [m]":[2.5E-9,],
   "SEI growth activation energy [J.mol-1]":[38000,],
   # Solvent consumption sub-model
   "Initial electrolyte excessive amount ratio":[ 2.0,], # set to <1 for DryOut=Off 
   "Current solvent concentration in the reservoir [mol.m-3]":[4541.0,],
   "Current electrolyte concentration in the reservoir [mol.m-3]":[1000,],
   "Ratio of Li-ion concentration change in electrolyte consider solvent consumption":[1.0,],
   # DFN parameter
   "Upper voltage cut-off [V]":[4.21,],
   "Lower voltage cut-off [V]":[2.49,],

   # interstitial-diffusion limited
   'Inner SEI lithium interstitial diffusivity [m2.s-1]':[1e-12,],    
   'Lithium interstitial reference concentration [mol.m-3]':[15,],
   # ec-reaction limited
   'EC diffusivity [m2.s-1]':[5e-21,],
   'SEI kinetic rate constant [m.s-1]':[1e-12,], 
   'EC initial concentration in electrolyte [mol.m-3]':[4541.0,],
   # LiP and coupling with SEI:
   "Dead lithium decay constant [s-1]":[ 1e-7,],             # default: 1e-6
   'Lithium plating kinetic rate constant [m.s-1]':[3E-10,], # default: 1e-9
   # Crack model
   "Negative electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   "Positive electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   # make it simple for now,], but may want to have T dependency in the future
   "Negative electrode cracking rate":[ 3.9e-22,],   # default: function, ~3.9e-20
   "Positive electrode cracking rate":[ 3.9e-22,],   # default: function, ~3.9e-20
   "Negative electrode volume change":[ 0.0,],
   "Positive electrode volume change":[ 0.0,],
   "Initial Neg SOC":[0.850],    #list(np.linspace(0.84,0.90,6)),
   "Initial Pos SOC":[0.2705], # list(np.linspace(0.22,0.27,6)),

}
Para_dict_list = []
recursive_scan(Para_dict_list,Para_dict_All, list(Para_dict_All.keys()), {})
print(f"Total scan case is {len(Para_dict_list)}")

Total scan case is 1


In [10]:
# manually define input para
# define experiments
V_max = 4.2;        
V_min = 2.5; 
exp_AGE_text = [(f"Discharge at 1 C until {V_min} V", 
        f"Charge at 0.3 C until {V_max} V (5 minute period)", 
        f"Hold at {V_max} V until C/100 (5 minute period)"),  ]
# step index for ageing
step_AGE_CD =0;   step_AGE_CC =1;   step_AGE_CV =2;

exp_RPT_text = [ (f"Discharge at 0.1C until {V_min} V",  
        "Rest for 1 hours (5 minute period)",  
        f"Charge at 0.1C until {V_max} V (5 minute period)" ) ]
# step index for RPT
step_RPT_CD = 0;  step_RPT_RE =1;   step_RPT_CC = 2;  

exp_text_list = [exp_AGE_text, exp_RPT_text,];
cycle_no = -1; 
exp_index_pack = [cycle_no,step_AGE_CD,step_AGE_CC,step_AGE_CV,
   step_RPT_CD,step_RPT_RE , step_RPT_CC ];
keys_loc_RPT = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode roughness ratio",
    "CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    "CDend Negative electrode roughness ratio",
    "CDend Total SEI on cracks thickness [m]",
    "REend Total SEI on cracks thickness [m]",
]
keys_tim_RPT = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "RE Terminal voltage [V]",
]
keys_cyc_RPT = [   # default: CDend
    "Discharge capacity [A.h]",
    "CDend Loss of capacity to lithium plating [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Loss of capacity to SEI on cracks [A.h]",
    "CDend X-averaged total SEI on cracks thickness [m]",
    "CDend X-averaged negative electrode roughness ratio",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode SOC", 
    "CDend Negative electrode SOC",
    "CDsta Positive electrode SOC", 
    "CDend Positive electrode SOC",
    "CDend Negative electrode capacity [A.h]",
    "CDend Positive electrode capacity [A.h]",
    "CDend Throughput capacity [A.h]",
]

keys_loc_AGE = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode roughness ratio",
    "CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    "CDend Negative electrode roughness ratio",
    "CDend Total SEI on cracks thickness [m]",
    "CDend Electrolyte diffusivity [m2.s-1]",
    "CDend Electrolyte conductivity [S.m-1]",
]
keys_tim_AGE = [];
keys_cyc_AGE = [];
keys_all_RPT = [keys_loc_RPT,keys_tim_RPT,keys_cyc_RPT];
keys_all_AGE = [keys_loc_AGE,keys_tim_AGE,keys_cyc_AGE];
keys_all = [keys_all_RPT,keys_all_AGE];

# Main code to run the model
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P2R7'; 
#BasicPath=os.getcwd()
Target  = '/b0_single/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target);
book_name_xlsx = 'a0s.xlsx';

sheet_name_xlsx = 'Results';

Path_pack = [BasicPath,Target,book_name_xlsx,sheet_name_xlsx,];

# Write the head for excel file:
head_list = list(Para_dict_list[0].keys())
head_list.insert(0,"Index");
head_list.insert(1,"Dry out");
head_list.extend([ "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error"])
write_excel_xlsx(BasicPath + Target+book_name_xlsx, sheet_name_xlsx, [head_list])


Successfully create a excel file


In [11]:
Para_dict_i = Para_dict_list[0]
index_xlsx = 1
ModelTimer = pb.Timer()
Scan_i = int(index_xlsx)
print('Start Now! Scan %d.' % Scan_i)  

# Un-pack data:
[cycle_no,step_AGE_CD,step_AGE_CC,step_AGE_CV,
    step_RPT_CD,step_RPT_RE , step_RPT_CC ] = exp_index_pack;
[exp_AGE_text, exp_RPT_text,] = exp_text_list;
[BasicPath,Target,book_name_xlsx,sheet_name_xlsx,] = Path_pack
CyclePack,Para_0 = Para_init(Para_dict_i)
[Total_Cycles,Cycle_bt_RPT,Update_Cycles,RPT_Cycles,
    Temper_i,Temper_RPT,mesh_par,submesh_strech,model_options] = CyclePack;
[keys_all_RPT,keys_all_AGE] = keys_all
str_exp_AGE_text  = str(exp_AGE_text);
str_exp_RPT_text  = str(exp_RPT_text);

# define experiment
Experiment_Long   = pb.Experiment( exp_AGE_text * Update_Cycles  )  
Experiment_RPT    = pb.Experiment( exp_RPT_text * RPT_Cycles     ) 
Experiment_Breakin= pb.Experiment( exp_RPT_text * RPT_Cycles     )

#####  index definition ######################
Small_Loop =  int(Cycle_bt_RPT/Update_Cycles);   
SaveTimes = int(Total_Cycles/Cycle_bt_RPT);   

# initialize my_dict for outputs
my_dict_RPT = {}
for keys in keys_all_RPT:
    for key in keys:
        my_dict_RPT[key]=[];
my_dict_AGE = {}; 
for keys in keys_all_AGE:
    for key in keys:
        my_dict_AGE[key]=[];
my_dict_RPT["Cycle_RPT"] = []; my_dict_AGE["Cycle_AGE"] = []; 
Cyc_Update_Index     =[]; 
        
# update 220924: merge DryOut and Int_ElelyExces_Ratio
temp_Int_ElelyExces_Ratio =  Para_0["Initial electrolyte excessive amount ratio"] 
ce_EC_0 = Para_0['EC initial concentration in electrolyte [mol.m-3]'] # used to calculate ce_EC_All
if temp_Int_ElelyExces_Ratio < 1:
    Int_ElelyExces_Ratio = -1;
    DryOut = "Off";
else:
    Int_ElelyExces_Ratio = temp_Int_ElelyExces_Ratio;
    DryOut = "On";
print(f"Scan {Scan_i}: DryOut = {DryOut}")
if DryOut == "On":  
    mdic_dry,Para_0 = Initialize_mdic_dry(Para_0,Int_ElelyExces_Ratio)
else:
    mdic_dry ={}

Start Now! Scan 1.
Scan 1: DryOut = On
Initial electrolyte amount is  10.735436400000001 mL


In [12]:
Model_0,Sol_0,Call_Breakin = Run_Breakin(
    model_options, Experiment_Breakin, 
    Para_0, mesh_par, submesh_strech)

In [14]:
Model_0.variables.search('capacity')

Discharge capacity [A.h]
Loss of capacity to SEI [A.h]
Loss of capacity to SEI on cracks [A.h]
Loss of capacity to dead lithium plating [A.h]
Loss of capacity to lithium plating [A.h]
Negative electrode capacity [A.h]
Positive electrode capacity [A.h]
Throughput capacity [A.h]
Total capacity lost to side reactions [A.h]


In [15]:
Sol_0["Positive electrode capacity [A.h]"].entries

array([8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231852,
       8.73231852, 8.73231852, 8.73231852, 8.73231852, 8.73231

In [16]:
Sol_0["Negative electrode capacity [A.h]"].entries

array([5.82761507, 5.82761507, 5.82761507, 5.82761507, 5.82761507,
       5.82761507, 5.82761507, 5.82761507, 5.82761507, 5.82761507,
       5.82761507, 5.82761507, 5.82761507, 5.82761507, 5.82761507,
       5.82761507, 5.82761507, 5.82761507, 5.82761507, 5.82761507,
       5.82761507, 5.82761507, 5.82761507, 5.82761507, 5.82761507,
       5.82761507, 5.82761507, 5.82761507, 5.82761507, 5.82761507,
       5.82761507, 5.82761507, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761506,
       5.82761506, 5.82761506, 5.82761506, 5.82761506, 5.82761